# 案例


应用场景：

本文使用决策树对**web站点的用户在线浏览行为**及**最终购买行为（选择的服务类型或者用户类型）进行预测。**

每个用于的在线浏览行为信息包括：每个用户的来源网站、用户的ip位置、是否阅读FAQ、浏览网页数目。

目标分类为用户类型：

>游客、基本用户、高级用户

在建立决策树时，我们先要懂得一个概念，叫属性的分类重要性，就是某个属性的出现，对目标结果能带来多大的信息。

属性的重要程度是根据样本数据集使用该属性进行划分子集后，集合的纯度增加了多少来决定。

我们以用户在线浏览信息为例，如果阅读过FAQ的用户全部都是高级用户，没有阅读过FAQ的都是基本用户，则是否阅读过FAQ这个属性就非常重要。

因为通过FAQ属性划分子集后，产生的两个子集，非常“纯”。


决策树的建立过程是先找出最重要的分类属性，再找出第二重要的分类属性。以此建立树的层次。

| 算法        | 支持模型   |  树结构  |   特征选择 | 连续值处理|  缺失值处理 |  剪枝 | 
| --------   | -----  | ----  |----  |----  |----  |----  |
|ID3 |分类 |多叉树 |信息增益| 不支持  |不支持 | 不支持 |
|C4.5 |分类 |多叉树| 信息增益比 |支持 | 支持 | 支持 |
|CART |分类，回归| 二叉树| 基尼系数，均方差| 支持 | 支持 | 支持 |

# 构建数据集


读者可以自己去获取自家公司的用户行为记录，这里给出一个简单的数据集

下面的数据为每个用户的来源网站、位置、是否阅读FAQ、浏览网页数目、以及用户类型（None为游客，Basic为基本用户，Premium为高级用户）。

最后一列属性为用户类型，就是我们想要预测的分类结果

In [1]:
my_data=[['slashdot','USA','yes',18,'None'],
         ['google','France','yes',23,'Premium'],
         ['digg','USA','yes',24,'Basic'],
         ['kiwitobes','France','yes',23,'Basic'],
         ['google','UK','no',21,'Premium'],
         ['(direct)','New Zealand','no',12,'None'],
         ['(direct)','UK','no',21,'Basic'],
         ['google','USA','no',24,'Premium'],
         ['slashdot','France','yes',19,'None'],
         ['digg','USA','no',18,'None'],
         ['google','UK','no',18,'None'],
         ['kiwitobes','UK','no',19,'None'],
         ['digg','New Zealand','yes',12,'Basic'],
         ['slashdot','UK','no',21,'None'],
         ['google','UK','yes',18,'Basic'],
         ['kiwitobes','France','yes',19,'Basic']]

# 决策树前的准备工作


有了样本数据集，我们就可以用来构建决策树了。

在构建决策树的过程中：

①首先我们要对决策树上的点创建类

②然后要能够根据属性划分成多个子数据集

③还要能计算划分子数据集后的信息增益，用信息增益来判断这个属性的重要性

④然后选择最重要的属性建第一层树。


**1、为决策树上的点创建类**


In [2]:
# 建立决策树上的节点类
class decisionnode:
    def __init__(self,col=-1,value=None,results=None,tb=None,fb=None):
        self.col=col                #待检测条件所属的列索引。即当前是对第几列数据进行分类
        self.value=value            #为使结果为true，当前列必须匹配的值
        self.results=results        #如果当前节点时叶节点，表示该节点的结果值，如果不是叶节点，为None
        self.tb=tb                  #判断条件为true后的子节点
        self.fb=fb                  #判断调节为false后的子节点

**2、数据集划分功能**

这里我们完成根据属性划分数据集的功能。一般对于字符串型的数据，我们将数据集分成等于和不等于两个子集。

对数值型属性，我们分成大于和小于两个子集。

需要注意的是，在样本数据集中，每一行为一个对象，每一列为一种属性。样本数据集以矩阵的形式存在。

In [3]:
# 根据某一属性对数据集合进行拆分，能够处理数值型数据或名词性数据。其实决策树只能处理离散型数据，对于连续性数据也是划分为范围区间块
# rows样本数据集，column要匹配的属性列索引，value指定列上的数据要匹配的值
def divideset(rows,column_index,column_value):
    # 定义一个函数，令其告诉我们数据行属于第一组（返回值为true）还是第二组（返回值false）
    split_function=None
    if isinstance(column_value,int) or isinstance(column_value,float):
        split_function=lambda row:row[column_index]>=column_value   #按大于、小于区分
    else:
        split_function=lambda row:row[column_index]==column_value   #按等于、不等于区分

    # 将数据集拆分成两个子集，并返回
    set1=[row for row in rows if split_function(row)]
    set2=[row for row in rows if not split_function(row)]
    return (set1,set2)

# 信息增益的计算

完成了拆分子集，就可以计算拆分后的信息增益了。

在计算信息增益时，我们不适用纯度，而使用凌乱度，意义相同。凌乱度越大越不好。

原样本集划分成多个子集后，每个子集的凌乱度我们可以由基尼不纯度的大小或熵的大小来代替。

那一个原数据集和多个子集之间如何比较凌乱度呢。这除了和每个子集的凌乱度有关外，还与子集的大小有关。

因为如果其中一个子集样本数非常少，即使凌乱度非常低也不能代表什么。

划分后的多个子集的总体凌乱度我们设定为**m=p1*m1+p2*m2+p3*m3。**其中p1、p2、p3为每个子集所占的比例，m1、m2、m3为每个子集的凌乱度。

用没有划分子集前的原样本数据集的凌乱度减去多个子集的总体凌乱度，就是信息增益。信息增益越大，证明该属性对分类的重要程度越大。

这里每个子集使用基尼不纯度或熵来代表凌乱度。


**1、基尼不纯度**

>子集计算基尼不纯度，即随机放置的数据项出现于错误分类中的概率。以此来评判属性对分类的重要程度。

$$Gini(p)=∑p_k (1−p_k )=1−∑ p^2_k  $$

换句话说，就是统计集合中所有分类的概率两两积的和。

比如下面的rows是通过一个属性划分的一个子集。我们肯定希望这个子集中尽可能都是同一种分类。

这样这个子集的纯度才能够高。如何看纯度够不够高呢。

比如这个集合有包含了3个分类，分类1的比例是0.1，分类2的比例是0.1，分类3的比例是0.8。这3个比例的两两积的和为0.1*0.1+0.1*0.8+0.1*0.8 = 0.17

而如果这三个分类中，分类1的比例为0.3，分类2的比例为0.3，分类3的比例为0.4，则这3个比例的两两积的和为0.3*0.3+0.3*0.4+0.3*0.4=0.33，是不是比上面的大，表明纯度小。

为什么会存在这种大小关系呢？自己百度吧。

In [4]:
#rows样本数据集
def giniimpurity(rows):
    total=len(rows)
    counts=uniquecounts(rows)
    imp=0
    for k1 in counts:
        p1=float(counts[k1])/total
        for k2 in counts:
            if k1==k2: continue
            p2=float(counts[k2])/total
            imp+=p1*p2
    return imp

# 统计集合rows中每种分类的样本数目。（样本数据每一行数据的最后一列记录了分类结果）。rows样本数据
def uniquecounts(rows):
    results={}
    for row in rows:
        # 目标结果在样本数据最后一列
        r=row[len(row)-1]
        if r not in results: results[r]=0
        results[r]+=1
    return results

**2、熵**

>熵，即遍历所有可能的结果之后得到的p(x)log(p(x))之和。也可以以此评判属性对分类的重要程度。

In [5]:
#rows样本数据集
def entropy(rows):
    from math import log
    log2=lambda x:log(x)/log(2)
    results=uniquecounts(rows)
    # 此处开始计算熵的值
    ent=0.0
    for r in results.keys():
        p=float(results[r])/len(rows)
        ent=ent-p*log2(p)
    return ent

# 对各种可能的目标结果（选择的服务类型）进行计数（样本数据每一行数据的最后一列记录了目标结果）。rows样本数据
def uniquecounts(rows):
    results={}
    for row in rows:
        # 目标结果在样本数据最后一列
        r=row[len(row)-1]
        if r not in results: results[r]=0
        results[r]+=1
    return results

## 构建决策树 ##

当能判断哪个属性对分类更重要我们就可以构建决策树了，先选出最重要的属性，作为根节点，再将数据集划分成两个子集，并分别在子集中选出其次重要的属性，做为左右子树的根节点，并以此递推下去。

通过每一次的属性查询，我们就知道了该找那个属性作为节点，以及该如果进行左右子树的划分。

buildtree函数输入为样本数据集，输出为决策树。

> 所谓的输出为决策树，就是输出的为根节点。根节点就表示决策树。


In [6]:
# 构建决策树.scoref为信息增益的计算函数
def buildtree(rows,scoref=entropy):
    if len(rows)==0: return decisionnode()
    current_score=scoref(rows)

    # 定义一些变量以记录最佳拆分条件
    best_gain=0.0
    best_criteria=None
    best_sets=None

    column_count=len(rows[0])-1
    for col in range(0,column_count):    #遍历每一列（除最后一列，因为最后一列是目标结果）
        # 在当前列中生成一个由不同值构成的序列
        column_values={}
        for row in rows:
            column_values[row[col]]=1
        # 接下来根据这一列中的每个值，尝试对数据集进行拆分
        for value in column_values.keys():
            (set1,set2)=divideset(rows,col,value)

            # 计算信息增益
            p=float(len(set1))/len(rows)
            gain=current_score-p*scoref(set1)-(1-p)*scoref(set2)
            if gain>best_gain and len(set1)>0 and len(set2)>0:   #找到信息增益最大的分类属性
                best_gain=gain
                best_criteria=(col,value)
                best_sets=(set1,set2)
    # 创建子分支
    if best_gain>0:
        trueBranch=buildtree(best_sets[0])   #创建分支
        falseBranch=buildtree(best_sets[1])  #创建分支
        return decisionnode(col=best_criteria[0],value=best_criteria[1],tb=trueBranch,fb=falseBranch)  #返回决策树节点
    else:
        return decisionnode(results=uniquecounts(rows))

# 绘制决策树

In [7]:
from PIL import Image, ImageDraw

# 获取树的显示宽度
def getwidth(tree):
    if tree.tb==None and tree.fb==None: return 1
    return getwidth(tree.tb)+getwidth(tree.fb)

# 获取树的显示深度（高度）
def getdepth(tree):
    if tree.tb==None and tree.fb==None: return 0
    return max(getdepth(tree.tb),getdepth(tree.fb))+1

# 绘制树形图
def drawtree(tree,jpeg='tree.jpg'):
    w=getwidth(tree)*100
    h=getdepth(tree)*100+120

    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)

    drawnode(draw,tree,w/2,20)  #根节点坐标
    img.save(jpeg,'JPEG')

# 迭代画树的节点
def drawnode(draw,tree,x,y):
    if tree.results==None:
        # 得到每个分支的宽度
        w1=getwidth(tree.fb)*100
        w2=getwidth(tree.tb)*100

        # 确定此节点所要占据的总空间
        left=x-(w1+w2)/2
        right=x+(w1+w2)/2

        # 绘制判断条件字符串
        draw.text((x-20,y-10),str(tree.col)+':'+str(tree.value),(0,0,0))

        # 绘制到分支的连线
        draw.line((x,y,left+w1/2,y+100),fill=(255,0,0))
        draw.line((x,y,right-w2/2,y+100),fill=(255,0,0))

        # 绘制分支的节点
        drawnode(draw,tree.fb,left+w1/2,y+100)
        drawnode(draw,tree.tb,right-w2/2,y+100)
    else:
        txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
        draw.text((x-20,y),txt,(0,0,0))


![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/6961712b9621ec20a98a212dc2c1b57b.png)

# 使用决策树对新的待测数据进行分类

In [8]:
# 对新的观测数据进行分类。observation为观测数据。tree为建立好的决策树
def classify(observation,tree):
    if tree.results!=None:
        return tree.results
    else:
        v=observation[tree.col]
        branch=None
        if isinstance(v,int) or isinstance(v,float):
            if v>=tree.value: branch=tree.tb
            else: branch=tree.fb
        else:
            if v==tree.value: branch=tree.tb
            else: branch=tree.fb
        return classify(observation,branch)

# 剪枝操作

在决策树创建时，由于数据中的噪声和离群点，许多分支反应的是训练数据中的异常，或者构建决策树时选取的阈值较小，造成构造的决策树特别复杂。

这些都导致决策树对训练数据的分类效果很好，但是对未知数据的分类效果不理想，也就是过拟合现象。

我们通过剪支方法处理这种过拟合数据问题。通常，这种方法使用统计量剪掉最不可靠的分支，一颗未剪枝的树和剪支的树对比如图。剪支后树更小，更简单，判断更快更好。

如图中，比如属性A1的值为no时正常情况都应该属于分类B，但是由于噪声原因，存在了几个属性A1的值为no的，却属于分类A的对象。所以在决策树时就会多出一条分支。

![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/d045c83b20bb71fa6ffbf373b69baded.png)

剪支分为先剪支和后剪支。后剪支更常用，就是先完整的构建一个树，再通过删除节点的分支并用树叶替换它而剪掉给定节点上的子树。

剪支的过程就是对具有相同父节点的一组节点进行检查，判断如果将其合并，熵的增加量是否会小于某个指定的阈值。

如果确实如此，则这些叶节点会被合并成一个单一的节点，合并后的新节点包含了所有可能的结果值。

这种做法有助于避免过度拟合的情况，也使得根据决策树做出的预测结果，不至于比从数据集中得到的实际结论还要特殊。

CART树的剪枝算法中使用的是用交叉验证来检验剪枝后的预测能力，选择泛化预测能力最好的剪枝后的数作为最终的CART树。

In [9]:
# 决策树剪枝。(因为有些属性的分类产生的熵值的差太小，没有区分的必要)，mingain为门限。
# 为了避免遇到大小台阶的问题（子树分支的属性比较重要），所以采取先建树，再剪支的方式
def prune(tree,mingain):
    # 如果分支不是叶节点，则对其进行剪枝操作
    if tree.tb.results==None:
        prune(tree.tb,mingain)
    if tree.fb.results==None:
        prune(tree.fb,mingain)

    # 如果两个自分支都是叶节点，则判断他们是否需要合并
    if tree.tb.results!=None and tree.fb.results!=None:
        # 构建合并后的数据集
        tb,fb=[],[]
        for v,c in tree.tb.results.items():
            tb+=[[v]]*c
        for v,c in tree.fb.results.items():
            fb+=[[v]]*c

        # 检查熵的减少情况
        delta=entropy(tb+fb)-(entropy(tb)+entropy(fb)/2)

        if delta<mingain:
            # 合并分支
            tree.tb,tree.fb=None,None
            tree.results=uniquecounts(tb+fb)

# 对于缺失数据的情况

对包含缺失数据的新的待测数据进行分类。会在逐层分类到缺失属性层时，不知道该往哪个方向继续判断。

这时可以支同时查询每个分支，这样就有多个最终分类结果。最后计算多个分类的结果的加权结果。

In [10]:
def mdclassify(observation,tree):
    if tree.results!=None:
        return tree.results
    else:
        v=observation[tree.col]
        if v==None:
            tr,fr=mdclassify(observation,tree.tb),mdclassify(observation,tree.fb) #分别使用左右子树进行分类预测
            # 求统计结果
            tcount=sum(tr.values())
            fcount=sum(fr.values())
            tw=float(tcount)/(tcount+fcount)
            fw=float(fcount)/(tcount+fcount)
            result={}
            for k,v in tr.items(): result[k]=v*tw
            for k,v in fr.items(): result[k]=v*fw
            return result
        else:
            if isinstance(v,int) or isinstance(v,float):
                if v>=tree.value: branch=tree.tb
                else: branch=tree.fb
            else:
                if v==tree.value: branch=tree.tb
                else: branch=tree.fb
            return mdclassify(observation,branch)

# 对于数值型输出结果


如果输出结果不是分类而是数字，可以使用方差作为评价函数来取代熵或基尼不纯度

In [11]:
#计算数据集的统计方差
def variance(rows):
    if len(rows)==0: return 0
    data=[float(row[len(row)-1]) for row in rows]
    mean=sum(data)/len(data)
    variance=sum([(d-mean)**2 for d in data])/len(data)
    return variance

# 测试代码

In [13]:
if __name__=='__main__':  #只有在执行当前模块时才会运行此函数
    tree = buildtree(my_data)   #创建决策树
    print(tree)
    drawtree(tree,jpeg='treeview.jpg')  #画树形图
    prune(tree,0.1)   #剪支
    result = mdclassify(['google',None,'yes',None],tree)   #使用决策树进行预测，新数据包含缺失数据
    print(result)

{'Premium': 2.25, 'Basic': 0.25}


决策树的内容到这里就讲完了。下面我们来总结一下。

**决策树优点**

1、决策树易于理解和实现，人们在在学习过程中不需要使用者了解很多的背景知识，这同时是它的能够直接体现数据的特点，只要通过解释后都有能力去理解决策树所表达的意义。

2、对于决策树，数据的准备往往是简单或者是不必要的，而且能够同时处理数据型和常规型属性，在相对短的时间内能够对大型数据源做出可行且效果良好的结果。

3、易于通过静态测试来对模型进行评测，可以测定模型可信度；如果给定一个观察的模型，那么根据所产生的决策树很容易推出相应的逻辑表达式。

**决策树缺点**

1)对连续性的字段比较难预测。

2)对有时间顺序的数据，需要很多预处理的工作。

3)当类别太多时，错误可能就会增加的比较快。

4)一般的算法分类的时候，只是根据一个字段来分类。

# Regression Decision Tree：回归树

回归树总体流程类似于分类树，区别在于，回归树的每一个节点都会得一个预测值。

以年龄为例，该预测值等于属于这个节点的所有人年龄的平均值。

分枝时穷举每一个feature的每个阈值找最好的分割点，但衡量最好的标准不再是最大熵，而是最小化平方误差。

也就是被预测出错的人数越多，错的越离谱，平方误差就越大，通过最小化平方误差能够找到最可靠的分枝依据。

分枝直到每个叶子节点上人的年龄都唯一或者达到预设的终止条件(如叶子个数上限)，若最终叶子节点上人的年龄不唯一，则以该节点上所有人的平均年龄做为该叶子节点的预测年龄。

![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/29786c02147b1dc0b561affde5c89b0f.png)

![这里写图片描述](https://img-blog.csdnimg.cn/img_convert/747b69ff088cd1fd7f91957ff938ea50.png)

> 思考：选哪些特征属性参与决策树建模、哪些属性排在决策树的顶部，哪些排在底部，对属性的值该进行什么样的判断、样本属性的值缺失怎么办、如果输出不是分类而是数值能用么、对决策没有用或没有多大帮助的属性怎么办、什么时候使用决策树？